# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

###Not for grading

## Random Forests vs Decision Trees on 20 class Documents data

In [1]:
#@title Case Study Walkthrough
#@markdown  Random Forests vs Decision Trees on 20 class Documents data
from IPython.display import HTML

HTML("""<video width="320" height="240" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_2018_b7_hyd/preview_videos/rf_vs_dt.mp4">
</video>
""")

### Setup Steps:


In [ ]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}


In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}


In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "U3W16_CS_RFvsDT_22_Doc" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/AIML_DS_NEWSGROUPS_PICKELFILE.pkl")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:",submission_id)
        print("Date of submission: ",r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    
else:
  print ("Please complete Id and Password cells before running setup")


##Importing packages

In [ ]:
# Importing required Packages
import pickle
import re
import operator
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import math
import collections
import gensim
from nltk import ngrams
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the dataset

dataset = pickle.load(open('AIML_DS_NEWSGROUPS_PICKELFILE.pkl','rb'))
print(dataset.keys())

In [ ]:
# Print frequencies of dataset
print("Class : count")
print("--------------")
number_of_documents = 0
for key in dataset:
    print(key, ':', len(dataset[key]))

Next, let us split our dataset which consists of 1000 samples per class, into training and test sets. We use 950 samples from each class in the training set, and the remaining 50 in the test set. 

As a mental exercise you should try reasoning about why is it important to ensure a nearly equal distribution of classes in your training and test sets. 

In [ ]:
train_set = {}
test_set = {}

# Clean dataset for text encoding issues :- Very useful when dealing with non-unicode characters
for key in dataset:
    dataset[key] = [[i.decode('utf-8', errors='replace').lower() for i in f] for f in dataset[key]]
    
# Break dataset into 95-5 split for training and testing
n_train = 0
n_test = 0
for k in dataset:
    split = int(0.95*len(dataset[k]))
    train_set[k] = dataset[k][0:split]
    test_set[k] = dataset[k][split:-1]
    n_train += len(train_set[k])
    n_test += len(test_set[k])

## 1. Bag-of-Words

Let us begin our journey into text classification with one of the simplest but most commonly used feature representations for news documents - Bag-of-Words.

As you might have realized, machine learning algorithms need good feature representations of different inputs.  Concretely, we would like to represent each news article $D$ in terms of a feature vector $V$, which can be used for classification. Feature vector $V$ is made up of the number of occurrences of each word in the vocabulary.

Let us begin by counting the number of occurrences of every word in the news documents in the training set.

### 1.1 Word frequency

Let us try understanding the kind of words that appear frequently, and those that occur rarely. We now count the frequencies of words:

In [ ]:
def frequency_words(train_set):
    frequency = defaultdict(int)

    for key in train_set:
        for f in train_set[key]:
            # Find all words which consist only of capital and lowercase characters and are between the length of 2-9.
            # We ignore all special characters such as !.$ and words containing numbers
            words = re.findall(r'(\b[A-Za-z][a-z]{2,9}\b)', ' '.join(f))
            for word in words:
                frequency[word] += 1
    return frequency

In [ ]:
frequency_of_words = frequency_words(train_set)

sorted_words = sorted(frequency_of_words.items(), key=operator.itemgetter(1), reverse=True)
print("Top-10 most frequent words:")
for word in sorted_words[:10]:
    print(word)

print('----------------------------')
print("10 least frequent words:")
for word in sorted_words[-10:-1]:
    print(word)

Next, we attempt to plot a histogram of the counts of various words in descending order. 

Could you comment on the relationship between the frequency of the most frequent word to the second frequent word? 
And what about the third most frequent word?

(Hint - Check the relative frequencies of the first, second and third most frequent words)

(After answering, you can visit https://en.wikipedia.org/wiki/Zipf%27s_law for further Reading)

In [ ]:
%matplotlib inline  

fig = plt.figure()
fig.set_size_inches(20,10)

plt.bar(range(len(sorted_words[:100])), [v for k, v in sorted_words[:100]] , align='center')
plt.xticks(range(len(sorted_words[:100])), [k for k, v in sorted_words[:100]])
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

### 1.2 Pre-processing to remove most and least frequent words

We can see that different words appear at different frequencies.

The most common words appear in almost all documents. Hence, for a classification task, having information about those words' frequencies does not matter much since they appear frequently in every type of document. To get a good feature representation, we eliminate them since they do not add too much value.

Additionally, notice how the least frequent words appear so rarely that they might not be useful either.

Let us pre-process our news articles now to remove the most frequent and least frequent words by thresholding their counts: 

In [ ]:
def cleaning_vocabulary_words(list_of_grams):
    valid_words = defaultdict(int)

    print('Number of words before preprocessing:', len(list_of_grams))

      # Ignore the 25 most frequent words, and the words which appear less than 100 times
    ignore_most_frequent = 25
    freq_thresh = 100
    feature_number = 0
    for word, word_frequency in list_of_grams[ignore_most_frequent:]:
        if word_frequency > freq_thresh:
            valid_words[word] = feature_number
            feature_number += 1
        elif '_' in word:
            valid_words[word] = feature_number
            feature_number += 1

    print('Number of words after preprocessing:', len(valid_words))

    vector_size = len(valid_words)
    return valid_words, vector_size

In [ ]:
valid_words, number_of_words = cleaning_vocabulary_words(sorted_words)

### 1.3 Bag-of-Words representation

The simplest way to represent a document $D$ as a vector $V$ would be to now count the relevant words in the document. 

For each document, make a vector of the count of each of the words in the vocabulary (excluding the words removed in the previous step - the "stopwords").

In [ ]:
def convert_to_BoW(dataset, number_of_documents):
    bow_representation = np.zeros((number_of_documents, number_of_words))
    labels = np.zeros((number_of_documents, 1))
    
    i = 0
    for label, class_name in enumerate(dataset):
        
        # For each file
        for f in dataset[class_name]:
            
            # Read all text in file
            text = ' '.join(f).split(' ')
            
            # For each word
            for word in text:
                if word in valid_words:
                    bow_representation[i, valid_words[word]] += 1
            
            # Label of document
            labels[i] = label
            
            # Increment document counter
            i += 1
    
    return bow_representation, labels

# Convert the dataset into their bag of words representation treating train and test separately
train_bow_set, train_bow_labels = convert_to_BoW(train_set, n_train)
test_bow_set, test_bow_labels = convert_to_BoW(test_set, n_test)

## Classification with Decision tree

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier()

In [ ]:
%%time

tree.fit(train_bow_set, train_bow_labels)

##Accuracy on Training data

In [ ]:
accuracy_score(tree.predict(train_bow_set), train_bow_labels)

In [ ]:
%%time
y_pred = tree.predict(test_bow_set)

## Accuracy on Test data

In [ ]:
accuracy_score(y_pred, test_bow_labels)

## Classification with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
%%time
rf.fit(train_bow_set, train_bow_labels)

##Accuracy on Train data

In [ ]:
accuracy_score(rf.predict(train_bow_set), train_bow_labels)

## Accuracy on Test data

In [ ]:

%%time
y_pred = rf.predict(test_bow_set)

In [ ]:
accuracy_score(y_pred, test_bow_labels)

## Change in Random Forest Accuracy with an increase in Estimators

In [ ]:
training_acc = []
test_acc = []
for i in range(1,101, 10):
    rf = RandomForestClassifier(n_estimators=i, random_state=0)
    rf.fit(train_bow_set, train_bow_labels)
    training_acc.append(accuracy_score(rf.predict(train_bow_set), train_bow_labels))
    test_acc.append(accuracy_score(rf.predict(test_bow_set), test_bow_labels))

In [ ]:
plt.plot(range(1,101, 10), training_acc, 'r', label='Training Accuracy')
plt.plot(range(1,101, 10), test_acc, 'b', label='Test Accuracy')
plt.xlabel('#Estimators')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

##Change in the accuracy of Random Forest with depth

### Below code block will take some time to complete the execution  

In [ ]:
d_training_acc = []
d_test_acc = []
for i in range(1,3001, 300):
    rf = RandomForestClassifier(n_estimators = 100, max_depth=i, random_state=0)
    rf.fit(train_bow_set, train_bow_labels)
    d_training_acc.append(accuracy_score(rf.predict(train_bow_set), train_bow_labels))
    d_test_acc.append(accuracy_score(rf.predict(test_bow_set), test_bow_labels))

In [ ]:
plt.plot(range(1,3001, 300), d_training_acc, 'r', label='Training Accuracy')
plt.plot(range(1,3001, 300), d_test_acc, 'b', label='Test Accuracy')
plt.xlabel('Depth of tree')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Random Forest')
plt.show()

##Change in the accuracy of  Decision Tree with depth

In [ ]:

dt_training_acc = []
dt_test_acc = []
for i in range(1,3001, 300):
    tree = DecisionTreeClassifier(max_depth=i)
    tree.fit(train_bow_set, train_bow_labels)
    dt_training_acc.append(accuracy_score(tree.predict(train_bow_set), train_bow_labels))
    dt_test_acc.append(accuracy_score(tree.predict(test_bow_set), test_bow_labels))

In [ ]:
plt.plot(range(1,3001, 300), dt_training_acc, 'r', label='Training Accuracy')
plt.plot(range(1,3001, 300), dt_test_acc, 'b', label='Test Accuracy')
plt.xlabel('Depth of tree')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Decision Tree')
plt.savefig('DT_depthvsAcc.png')
plt.show()

## Please answer the questions below to complete the experiment:

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]

In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook  { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")